In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from numpy.random import random as rand

import networkqit as nq
import networkx as nx

from networkqit import graph_laplacian as GL

import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import sympy as sp
import sympyprinting as spp
from sympy import init_printing

In [ ]:
n=sp.Symbol('n',integer=True)
pin=sp.Symbol('pin',real=True)
pout=sp.Symbol('pout',real=True)
a=sp.Symbol('a',real=True)
z=sp.Symbol('z',complex=True)
x,y=sp.Symbol('x',real=True),sp.Symbol('y',real=True)
c=sp.Symbol('c',integer=True)
t=sp.Symbol('t',complex=True)
d=sp.Symbol('d',integer=True)
eps=sp.Symbol('epsilon')

In [ ]:
# THIS IS THE MATHEMATICA VERSION USING MPMATH FOR HIGHER ACCURACY
import mpmath as mp
def Gamma(a,z0,z1): # This defined function has the same behaviour as Mathematica Gamma[a,z0,z1]
    from mpmath import gammainc
    return gammainc(a,a=z0,b=z1,regularized=False)

def scipyGamma(a,z0,z1):
    return (scipy.special.gammainc(a,z1)-scipy.special.gammainc(a,z0))*scipy.special.gamma(a)

In [ ]:
def fundamental_eq(n,pin,pout,t,z):
    feq = sp.simplify(t - sp.summation( sp.KroneckerDelta(c,n*pin/2)/(z-c-n*(pin+pout)/2*t ),(c,0,sp.oo)))
    return feq
    #return sp.lambdify((n,pin,pout,t,z),feq)

def mpmathfeq(n,pin,pout,t,z):
    #print(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5)
    Eq = t + (2.0**(n*(pin + pout)*t - z)*(-(n*pin))**(-(n*(pin + pout)*t) + z)*Gamma(n*(pin + pout)*t - z,0,-(n*pin)/2.))/mp.exp((n*pin)/2.)
    #print('t=',t,'Eq=',(float(Eq.real),float(Eq.imag)))
    return (float(Eq.real),float(Eq.imag))

def feqlapl(n,pin,pout,t,z):
    #print(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5)
    Eq = t + (2.0**(n*(pin + pout)*t - z)*(-(n*pin))**(-(n*(pin + pout)*t) + z)*Gamma(n*(pin + pout)*t - z,0,-(n*pin)/2.))/mp.exp((n*pin)/2.)
    #print('t=',t,'Eq=',(float(Eq.real),float(Eq.imag)))
    return (float(Eq.real),float(Eq.imag))

def scipyfeq(n,pin,pout,t,z):
    print(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5)
    Eq = t + (2.0**(n*(pin + pout)*t - z)*(-(n*pin))**(-(n*(pin + pout)*t) + z)*scipyGamma(n*(pin + pout)*t - z,0.0,-(n*pin)*0.5))/np.exp((n*pin)/2.)
    return (float(Eq.real),float(Eq.imag))

In [ ]:
init_printing()
fundamental_eq(n,pin,pout,t,z)
sp.solve(t+2/(n*pin+n*t*(pin+pout)-2*z),t)

In [ ]:
print(scipy.optimize.fsolve(lambda t : mpmathfeq(10,0.8,0.2,t[0]-1j*t[1],1),(0.01,0.01)))
print(scipy.optimize.root(lambda t : mpmathfeq(10,0.8,0.2,t[0]-1j*t[1],1),(0.01,0.01)).x)

In [ ]:
def mpmathfeq(n,pin,pout,t,z):
    Eq = t + (2**((n*(pin + pout)*t)/2. - z)*(-(n*(pin + pout)))**(-(n*(pin + pout)*t)/2. + z)*Gamma((n*(pin + pout)*t)/2. - z,0,-(n*(pin + pout))/2.))/mp.exp((n*(pin + pout))/2.)
    #print('t=',t,'Eq=',(float(Eq.real),float(Eq.imag)))
    return (float(Eq.real),float(Eq.imag))

#scipy.optimize.root(lambda t : mpmathfeq(10,0.8,0.2,t[0]+1j*t[1],1+1E-4j),(0.01,-1))

In [ ]:
N,B,pin,pout=100,2,0.8,0.8
allz=np.linspace(-20,20,5000)

import networkx as nx
eigs_l,eigs_a=[],[]
for i in range(0,100):
    A=nx.to_numpy_array(nx.planted_partition_graph(B,int(N/B),pin,pout))
    L=np.diag(A.sum(axis=0))-A
    eigs_l.append(scipy.linalg.eigvalsh(L))
eigs_a = np.array(eigs_a).flatten()
eigs_l = np.array(eigs_l).flatten()

In [ ]:
plt.figure(figsize=(18,5))
plt.hist(eigs_l,500,density=True,color='blue',alpha=0.4)
#plt.plot(allz,2/(N*np.pi)*(np.sqrt(N*(pin+pout)-allz**2)/(pin+pout)))
#plt.vlines(np.sqrt(2)*N*(pin+pout),ymin=0,ymax=0.05)
#plt.vlines(-np.sqrt(2*N*(pin+pout)/2),ymin=0,ymax=0.05)
plt.show()

In [ ]:
def mpmathfeq(n,pin,pout,t,z):
    Eq = t + (2**((n*(pin + pout)*t)/2. - z)*(-(n*(pin + pout)))**(-(n*(pin + pout)*t)/2. + z)*Gamma((n*(pin + pout)*t)/2. - z,0,-(n*(pin + pout))/2.))/mp.exp((n*(pin + pout))/2.)
    #print('t=',t,'Eq=',(float(Eq.real),float(Eq.imag)))
    return (float(Eq.real),float(Eq.imag))

T=[]
allz=np.linspace(0,N,N)
for i,z in enumerate(allz):
    t = scipy.optimize.fsolve(lambda t : mpmathfeq(N,pin,pout,t[0]+1j*t[1],z+0.001j),(0.1,-0.1))[1]
    T.append(t)
    print('\r',i,(i+1)/len(allz)*100,'%',end='')
T=np.array(T)
rho = -1/(np.pi*N)*(N/2*T+N/2*T)
plt.figure(figsize=(18,5))
plt.hist(eigs_l,500,density=True,color='blue',alpha=0.4)
plt.plot(allz,rho)
plt.show()